# 6. 작사가 인공지능 만들기

# 6-7. 프로젝트: 멋진 작사가 만들기

## 데이터 준비

In [1]:
# 데이터 준비
# wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
# unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  #lyrics 폴더에 압축풀기

## 데이터 읽어오기

In [2]:
# 데이터 읽어오기
# glob 모듈을 사용하면 파일 읽어오는 작업 하기 용이함
# glob을 활용하여 모든 txt파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장

import glob
import re       # 정규표현식을 위한 Regax 지원 모듈(문장 데이터 정돈에 필요)
import numpy as np       # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf   # 텐서플로우
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러 개의 txt 파일을 모두 읽어서 raw_corpus에 담김
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print(type(raw_corpus))        
print("데이터 크기: ", len(raw_corpus))
print("Examples:\n", raw_corpus[:10], end="\t")

<class 'list'>
데이터 크기:  187088
Examples:
 ['', '', 'All of this and more is for you', 'With love, sincerity and deepest care', 'My life with you I share', '', '', 'Ever since I met you, baby', "I've been wantin' to lay you down", "But it's so hard to get you"]	

## 데이터 정제

In [3]:
## 데이터 정제
## preprocess_sentence() 함수 활용
## 긴 문장은 과도한 padding을 갖게 하므로 제거.
## 그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외

In [4]:
for idx, sentence in enumerate(raw_corpus):
#     if len(sentence)
    if len(sentence) == 0: continue # 공백 건너뜀
    if sentence[-1] == ":": continue # 문장 시작이 ' 건너뜀
#     if sentence[-1] == "'": continue # 문장 끝이 ' 건너뜀
#     if sentence[0] == '"': continue # 문장 시작이 " 건너뜀
#     if sentence[-1] == '"': continue # 문장 끝이 " 건너뜀
#     if sentence[0] == '(': continue # 문장 끝이 ( 건너뜀
#     if sentence[-1] == ')': continue # 문장 끝이 ) 건너뜀
#     if sentence[0] == '[': continue # 문장 끝이 [ 건너뜀
#     if sentence[-1] == ']': continue # 문장 끝이 ] 건너뜀        
#     if len(sentence) <= 15 : continue
    
#     if idx >= 15: continue # 15개 넘어가는 문장 건너뜀 : <=15는 15이하여서 15개 이하 건너 뛰는 거 아닌가? 
    if idx >15: break # 일단 문장 10개에서 끝내
        
    print(sentence)

All of this and more is for you
With love, sincerity and deepest care
My life with you I share
Ever since I met you, baby
I've been wantin' to lay you down
But it's so hard to get you
Baby, when you never come around
Every day that you keep it away
It only makes me want it more
Ooh baby, just say the word
And I'll be at your door


In [5]:
print(type(sentence))

<class 'str'>


In [6]:
# split()
# result = preprocess_sentense
# ()
# len(result.split())

In [7]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter(구분문자)로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()
    

    
    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample    sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [8]:
# def preprocess_sentence_2(preprocess_sentence):
#     for result in preprocess_sentence(sentence):
#         if len(preprocess_sentence(sentence)) > 30: continue
#     return result

In [9]:
# print(preprocess_sentence(sentence))
# print(preprocess_sentence_2(preprocess_sentence))


In [10]:
# print(type(preprocess_sentence(sentence)))
# print(len(preprocess_sentence(sentence)[10]))

In [11]:
# split()
# result = preprocess_sentense
# ()
# len(result.split())

In [12]:
# 정제함수 활용하여 정제 데이터 구축
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue # 공백 건너뜀
    if sentence[-1] == ":": continue # 문장 시작이 ' 건너뜀
#     if sentence[-1] == "'": continue # 문장 끝이 ' 건너뜀
#     if sentence[0] == '"': continue # 문장 시작이 " 건너뜀
#     if sentence[-1] == '"': continue # 문장 끝이 " 건너뜀
#     if sentence[0] == '(': continue # 문장 끝이 ( 건너뜀
#     if sentence[-1] == ')': continue # 문장 끝이 ) 건너뜀
#     if sentence[0] == '[': continue # 문장 끝이 [ 건너뜀
#     if sentence[-1] == ']': continue # 문장 끝이 ] 건너뜀
# #     if len(sentence) >= 17 : continue

        
#     if len(preprocess_sentence(sentence)) > 15 : continue
        
#     if sentence < 30: continue # 15개 넘어가는 문장 건너뜀 # 이건 글자가 30자 미만인거고 단어가 30개 미만이려면?
        
    corpus.append(preprocess_sentence(sentence)) # corpus에 추가해라 만든 정제함수(preprocess_sentece)에 sentence를 넣어서

# 평가 데이터셋 분리

In [13]:
## 평가 데이터셋 분리
## tokenize() 함수로 데이터를 tensor로 변환한 후 
## sklearn 모듈의 train_test_split()함수를 사용해 훈련데이터와 평가데이터 분리
## 단어장의 크기는 12,000이상으로 설정, 총 데이터의 20%를 평가 데이터셋으로 사용
## enc_train, enc_val, dec_train, dec_val = <코드 작성>

In [14]:
def tokenize(corpus):  
    # 텐서플로우에서 제공하는 Tokenizer패키지 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000, # 전체 단어 개수
        filters=' ', # 별도로 전처리 로직을 추가할 수 있지만 이번에는 사용 안 함
        oov_token="<unk>" # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus) # 우리가 구축한 corpus로부터 Tokenize가 사전을 자동 구축

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋 구축
    tensor = tokenizer.texts_to_sequences(corpus)  # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding 메소드 제공
    # maxlen의 디폴트 값은 None. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰짐
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen= 15, padding='post') # 토큰화 15개 미만?

    print(tensor, tokenizer)
    return tensor, tokenizer


tensor, tokenizer = tokenize(corpus)


print(type(tensor))
print(type(tokenizer))
print('ok')

# tensor = [c for c in raw_corpus if c.count(" ") <13] # 어디다 넣는 건지.........


[[    2    24    19 ...     0     0     0]
 [    2    31    33 ...     0     0     0]
 [    2    13   102 ...     0     0     0]
 ...
 [    2    24     6 ...     0     0     0]
 [    4     5    35 ...    96   940     3]
 [    2    67 11220 ...     0     0     0]] <keras_preprocessing.text.Tokenizer object at 0x7fb26e2bfc50>
<class 'numpy.ndarray'>
<class 'keras_preprocessing.text.Tokenizer'>
ok


In [15]:
print(type(tensor))

<class 'numpy.ndarray'>


In [16]:
print(tensor[:3])

[[    2    24    19    42     8    98    26    28     7     3     0     0
      0     0     0]
 [    2    31    33     4 11330     8  3995   282     3     0     0     0
      0     0     0]
 [    2    13   102    31     7     5   998     3     0     0     0     0
      0     0     0]]


In [17]:
# 텐서데이터는 모두 정수로 이루어짐
# 이 숫자는 tokenizer에 구축된 단어 사전의 인덱스
# 단어 사전 어떻게 구축되었는지 확인

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 15: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : t


In [18]:
src_input = tensor[:, :-1] # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
tgt_input = tensor[:, 1:] # tensor에서 <satrt>를 잘라내서 타겟문장 생성

print(type(tensor))
print(src_input[0])
print(tgt_input[0])

<class 'numpy.ndarray'>
[ 2 24 19 42  8 98 26 28  7  3  0  0  0  0]
[24 19 42  8 98 26 28  7  3  0  0  0  0  0]


In [19]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1 # tokenizer가 구축된 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>!

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [20]:
# skelarn의 train_test_split 함수를 통해 train_data와 test_data분리  
# test_size 인자를 조절해주면 설정해준 값 만큼 test dataset 비율 조정 가능
# from sklearn.datasets import corpus
from sklearn.model_selection import train_test_split 


enc_train, enc_val, dec_train, dec_val = train_test_split(tensor, tensor, test_size=0.2, shuffle=True)


In [21]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print(enc_val.shape)
print(dec_val.shape)
print(enc_train[0])

Source Train: (140599, 15)
Target Train: (140599, 15)
(35150, 15)
(35150, 15)
[  2   7 176 124 543 528   3   0   0   0   0   0   0   0   0]


In [22]:
## 위 과정까지 올바르게 진행했을 경우 아래의 실행결과 확인 가능
# print("Source Train : ", enc_train.shape)
# print("Target Train : ", dec_train.shape)

# out:
# Source Train: (124960, 14)
# Target Train: (124960, 14)

## 만약 결과가 다르면 동일한 결과를 얻도록 해야함
## 학습데이터 개수가 124960 보다 크다면 step3다시 확인

In [23]:
# result = preprocess_sentence(sentence)
# len(result.split()) < 15
# print(len(result.split()))

In [24]:
print(tensor[12000])
print(corpus[12000])

[   2   10  184   75   95  120 9768  974 9769    3    0    0    0    0
    0]
<start> to yo back then ya shiverin tongue deliverin <end>


In [25]:
print(type(tensor.shape))
print(type(tokenizer))

<class 'tuple'>
<class 'keras_preprocessing.text.Tokenizer'>


In [26]:
# 생성된 텐서 데이터를 출력
print(tensor[1])

[    2    31    33     4 11330     8  3995   282     3     0     0     0
     0     0     0]


# Step 5. 인공지능 만들기

In [27]:
# 인공지능 만들기
# 모델의 embedding size와 hideen size를 조절하여 10epoch안에 val_loss값을 2.2 수준으로 줄일 수 잇는 모델 설계
# loss는 아래 제시된 loss함수 그대로 사용

# loss = tf.keras.losses.SparseCategoricalCrossentropy(
#     from_logits=True, reduction='none')

# generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

In [28]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 #embedding_size는 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
hidden_size = 1024 # hidden_size는 모델에 얼마나 많은 일꾼을 둘 것인가 정도로 이해하면 됨
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

print('ok')

ok


In [29]:
# model에 데이터 태우기

for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 7.72190688e-05, -2.56716594e-04, -7.82191055e-05, ...,
         -1.94737688e-04,  4.41184930e-05,  9.12978940e-05],
        [ 6.32292067e-05, -4.12559748e-04, -6.27389411e-04, ...,
         -3.93010472e-04, -2.68280390e-04,  1.04974548e-04],
        [ 1.51692555e-04, -3.38889018e-04, -7.05388724e-04, ...,
         -3.91828216e-04, -5.90193784e-04, -1.77217862e-06],
        ...,
        [ 6.47143344e-04,  4.30490501e-04, -9.33299714e-04, ...,
         -1.97086993e-04, -1.43020751e-03, -4.44302976e-04],
        [ 7.42186268e-04,  8.32634454e-04, -1.05870469e-03, ...,
          1.76934773e-04, -1.67344220e-03, -3.66209133e-04],
        [ 8.45349510e-04,  7.60403578e-04, -1.15285127e-03, ...,
          4.85686556e-04, -1.85262680e-03, -5.72123681e-04]],

       [[ 7.72190688e-05, -2.56716594e-04, -7.82191055e-05, ...,
         -1.94737688e-04,  4.41184930e-05,  9.12978940e-05],
        [ 1.38106305e-04, -1.88761813e-04, -2

In [30]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [31]:
# 모델 학습 시작
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 85s 124ms/step - loss: 3.6197
Epoch 2/10
686/686 [==============================] - 86s 125ms/step - loss: 3.1207
Epoch 3/10
686/686 [==============================] - 86s 126ms/step - loss: 2.9225
Epoch 4/10
686/686 [==============================] - 86s 126ms/step - loss: 2.7704
Epoch 5/10
686/686 [==============================] - 89s 129ms/step - loss: 2.6384
Epoch 6/10
686/686 [==============================] - 89s 129ms/step - loss: 2.5193
Epoch 7/10
686/686 [==============================] - 88s 128ms/step - loss: 2.4103
Epoch 8/10
686/686 [==============================] - 88s 128ms/step - loss: 2.3150
Epoch 9/10
686/686 [==============================] - 87s 127ms/step - loss: 2.2120
Epoch 10/10
686/686 [==============================] - 87s 127ms/step - loss: 2.1183


In [32]:
# generate_text 함수는 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행함

def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해 입력받은 init_sentence도 일단 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성 해야함
    while True:
        predict = model(test_tensor) # 입력받은 문장의 텐서를 입력
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] # 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됨
        
        # 모델이 새롭게 예측한 단어를 입력 문장 뒤에 붙여줌
        test_tensor = tf.concat([test_tensor, 
                                 tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        # 모델이 <end>를 예측했거나, max_len에 도달하지 않았으면 while루프를 또 돌면서 다음 단어를 예측
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
            
    generated = ""
              
    # 생성된 tensor안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
        
    return generated # 이것이 최종적으로 모델이 생성한 자연어 문장
print('ok')

ok


In [33]:
# 위 문장 생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> love")

'<start> love is a beautiful thing <end> '

In [34]:
# 위 문장 생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> peace")

'<start> peace to <unk> , <end> '

In [36]:
# 위 문장 생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> home")

'<start> home of the moon <end> '

# 총평

## 루브릭 평가  
### 1. 그럴듯한 문장으로 생성되는가?
- 잘 생성이 안되는 것도 있고 제대로 생성되는 문장도 있다. 아마 내가 뭘 잘못해서 그런 것 같은데 코드를 거의 복사 붙여넣기를 한 것이라 뭐가 어디서 잘못됐는지 못찾겠다.
- 아마 토큰화부분에서 maxlen을 지정해주지 않고 파이썬 코드를 통해 토큰화를 15개 미만으로 해야하는 것 같은데 어떻게 하는지 잘 모르겠다. 저번에 다른 사람이 이렇게 했다고 알려줘서 보고 해봤지만 잘 모르겠다.(내가 알려주신 분의 코드를 제대로 못 본 것 같다.)
- 그래도 작동이 되는건 아마 end부분이 제일 끝에 있는 것이 아니여서 maxlen을 지정해도 작동이 되는 것 같다..?
  
### 2. 특수문자  제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?
- 복사 붙여넣기 해서 빠진 부분은 없는 것 같다.
    
### 3. validation loss까 2.2이하로 낮아졌는가?
- 돌려보니 최종적으로 2.2이하로 낮아졌다.